Chào cả lớp,

Dựa vào Q-Learning trong file đính kèm, các em hãy cày đặt thuật toán SARSA nhé (slides Reinforcement Learning).

Các em có nhận xét so sánh gì về performance của Q-Learning và SARSA trên 3 env "FrozenLake-v1", "FrozenLake8x8-v1", và "Taxi-v3"? Hãy vẽ đồ thị minh họa hiệu năng của SARSA so với Q-Learning. Viết nhận xét trực tiếp vào file bài nộp ipynb. Đặt tên file BT5_MSSV.ipynb với MSSV của mình.

Deadline: 18/06/2023

Sau ngày 25/06/2023 sẽ không nhân thêm bài nộp mới.

In [1]:
import gym
import numpy as np
import random

In [2]:
env = gym.make('FrozenLake-v1')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
# Initialize Q-value table randomly
q_table = np.zeros((env.observation_space.n, env.action_space.n))
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [4]:
# Hyperparameters
gamma = 0.99
learning_rate = 0.1
max_epsilon = 1.0
min_epsilon = 0.01
epsilon_decay_rate = 0.005

num_episodes = 20000
num_steps_per_episode = 100

In [5]:
def q_learning(env, num_episodes, num_steps_per_episode, learning_rate, gamma, max_epsilon, min_epsilon, epsilon_decay_rate):
    q_table = np.zeros((env.observation_space.n, env.action_space.n))
    rewards_all = []

    #Loop qua cac episodes
    for episode in range(num_episodes):
        #Chon state
        state = env.reset()
        reward_episode = 0.0

        #Epsilon de chọn Q
        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-epsilon_decay_rate*episode)
        #Loop qua cac step
        for step in range(num_steps_per_episode):
            #Chon cac action dua vao uniform distribution
            exploration = random.uniform(0,1)
            if exploration < epsilon:
                action = env.action_space.sample()
            else:
                action = np.argmax(q_table[state, :])

            next_state, reward, done, info = env.step(action)

            #Sample dung Q(s',a')
            sample=reward + gamma * np.max(q_table[next_state,:])
            q_table[state, action] = q_table[state, action] * (1 - learning_rate) + learning_rate * sample

            #Chuyen sang trang thai tiep theo
            reward_episode += reward
            state = next_state

        rewards_all.append(reward_episode)
    print(f'Episode {episode} finished')
    return q_table, rewards_all

In [6]:
q_table, rewards_all = q_learning(env, num_episodes, num_steps_per_episode, learning_rate, gamma, max_epsilon, min_epsilon, epsilon_decay_rate)

Episode 19999 finished


In [7]:
q_table

array([[0.51401123, 0.4844628 , 0.48506094, 0.4884316 ],
       [0.25003028, 0.13462748, 0.2550257 , 0.44207576],
       [0.36510921, 0.25967556, 0.2619514 , 0.25643709],
       [0.12430718, 0.0010159 , 0.00127433, 0.00378917],
       [0.53046954, 0.31914557, 0.41726267, 0.32027132],
       [0.        , 0.        , 0.        , 0.        ],
       [0.14363479, 0.15350617, 0.30083176, 0.10576669],
       [0.        , 0.        , 0.        , 0.        ],
       [0.31157143, 0.40402404, 0.34262328, 0.56334499],
       [0.49901939, 0.61006844, 0.46381585, 0.35561691],
       [0.55237127, 0.3420443 , 0.32597511, 0.27517028],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.30941173, 0.56077623, 0.69919988, 0.47194702],
       [0.723095  , 0.83702376, 0.7218498 , 0.67403342],
       [0.        , 0.        , 0.        , 0.        ]])

In [8]:
sum(rewards_all)
sum(rewards_all[0:1000])
sum(rewards_all[1000:2000])
sum(rewards_all[2000:3000])
sum(rewards_all[9000:10000])

665.0

In [9]:
def play(env, q_table, render=False):
    state = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = np.argmax(q_table[state, :])
        next_state, reward, done, info = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            env.render()
            time.sleep(0.2)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [10]:
def play_multiple_times(env, q_table, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, q_table)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')

In [11]:
play_multiple_times(env, q_table, 1000)

Number of successes: 709/1000
Average number of steps: 36.95345557122708


# Sarsa

Em cài đặt sarsa dựa trên epsilon thường, epsilon-greedy chỉ khác ở cách chúng ta lấy mẫu

In [12]:
def sarsa(env, num_episodes, num_steps_per_episode, learning_rate, gamma, max_epsilon, min_epsilon, epsilon_decay_rate):
    q_table = np.zeros((env.observation_space.n, env.action_space.n))
    rewards_all = []

    #Loop qua cac episodes
    for episode in range(num_episodes):
        #Chon state
        state = env.reset()
        reward_episode = 0.0
        #Epsilon de chọn action a
        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-epsilon_decay_rate*episode)

        exploration = random.uniform(0,1)
        if exploration < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state, :])


        #Loop qua cac step
        for step in range(num_steps_per_episode):
            #s' next state tiep theo
            next_state, reward, done, info = env.step(action)
            #Chon cac next_action dua vao uniform distribution cua next_state
            exploration = random.uniform(0,1)
            if exploration < epsilon:
                next_action = env.action_space.sample()
            else:
                next_action = np.argmax(q_table[next_state, :])

            #Sample dung Q
            sample=reward + gamma * q_table[next_state,next_action] - q_table[state, action]
            q_table[state, action] += learning_rate * sample

            #Chuyen sang trang thai tiep theo
            reward_episode += reward
            state = next_state
            #action cho action tiep theo
            action = next_action

        rewards_all.append(reward_episode)
    print(f'Episode {episode} finished')
    return q_table, rewards_all

In [13]:
q_table, rewards_all = sarsa(env, num_episodes, num_steps_per_episode, learning_rate, gamma, max_epsilon, min_epsilon, epsilon_decay_rate)

Episode 19999 finished


In [14]:
q_table

array([[0.       , 0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       ],
       [0.00891  , 0.0098901, 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.18991  , 0.0909801],
       [0.       , 0.       , 0.       , 0.       ]])

In [15]:
play_multiple_times(env, q_table, 1000)

Number of successes: 0/1000
Average number of steps: nan


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


# Chạy trên các game

In [16]:
import time

FrozenLake-v1_qlearning

In [17]:
env = gym.make('FrozenLake-v1')
start=time.time()
q_table, rewards_all = q_learning(env, num_episodes, num_steps_per_episode, learning_rate, gamma, max_epsilon, min_epsilon, epsilon_decay_rate)
end=time.time()
print('Time:',end-start)
play_multiple_times(env, q_table, 1000)

Episode 19999 finished
Time: 74.00147533416748
Number of successes: 726/1000
Average number of steps: 36.46280991735537


FrozenLake-v1_sarsa

In [18]:
env = gym.make('FrozenLake-v1')
start=time.time()
q_table, rewards_all = sarsa(env, num_episodes, num_steps_per_episode, learning_rate, gamma, max_epsilon, min_epsilon, epsilon_decay_rate)
end=time.time()
print('Time:',end-start)
play_multiple_times(env, q_table, 1000)

Episode 19999 finished
Time: 53.15991139411926
Number of successes: 732/1000
Average number of steps: 36.84426229508197


FrozenLake88-v1_qlearning

In [19]:
env = gym.make('FrozenLake8x8-v1')
start=time.time()
q_table, rewards_all = q_learning(env, num_episodes, num_steps_per_episode, learning_rate, gamma, max_epsilon, min_epsilon, epsilon_decay_rate)
end=time.time()
print('Time:',end-start)
play_multiple_times(env, q_table, 1000)

Episode 19999 finished
Time: 71.94302535057068
Number of successes: 0/1000
Average number of steps: nan


FrozenLake88-v1_sarsa

In [20]:
env = gym.make('FrozenLake8x8-v1')
start=time.time()
q_table, rewards_all = sarsa(env, num_episodes, num_steps_per_episode, learning_rate, gamma, max_epsilon, min_epsilon, epsilon_decay_rate)
end=time.time()
print('Time:',end-start)
play_multiple_times(env, q_table, 1000)

Episode 19999 finished
Time: 53.96932649612427
Number of successes: 0/1000
Average number of steps: nan


Taxi-v3_qlearning

In [21]:
env = gym.make('Taxi-v3')
start=time.time()
q_table, rewards_all = q_learning(env, num_episodes, num_steps_per_episode, learning_rate, gamma, max_epsilon, min_epsilon, epsilon_decay_rate)
end=time.time()
print('Time:',end-start)
play_multiple_times(env, q_table, 1000)

Episode 19999 finished
Time: 96.78189778327942
Number of successes: 1000/1000
Average number of steps: 13.016


Taxi-v3_sarsa

In [22]:
env = gym.make('Taxi-v3')
start=time.time()
q_table, rewards_all = sarsa(env, num_episodes, num_steps_per_episode, learning_rate, gamma, max_epsilon, min_epsilon, epsilon_decay_rate)
end=time.time()
print('Time:',end-start)
play_multiple_times(env, q_table, 1000)

Episode 19999 finished
Time: 69.14348649978638
Number of successes: 1000/1000
Average number of steps: 12.989


# **Nhận Xét**

*   Cả 2 thuật toán đều hội tụ dần về phương án tối ưu
*   Thuật toán Sarsa chạy nhanh hơn Qlearning (thời gian thực hiện nhanh hơn)
*   Số lần chơi thành công và số bước chơi trung bình qua 3 map của 2 thuật toán Q Learning và SARSA là tương đương nhau
*   Ở game taxi, cả 2 thuật toán đều cùng đã hội tụ về phương án tối ưu




